In [38]:
import requests
from io import StringIO
from Bio import SeqIO
import pandas as pd


Initially the reference gene names were obtained by extracting IDs from the assembly_2 FASTA file. 

In [ ]:
def extract_gene_names(fasta_file):
    gene_names = []

    for sequence in SeqIO.parse(fasta_file, "fasta"):
        
        # Assuming gene name is the second word in the description line
        gene_name = sequence.description.split()[1]
        
        # If gene We first extracted all the reference gene names from the assembly_2 fasta file. name exists, add it to the list
        if gene_name:
            gene_names.append(gene_name)

    return gene_names

fasta_file_path = 'data/assembly_2.prot.fa'

# Extract gene names from the FASTA file
ref_gene_names = extract_gene_names(fasta_file_path)

# Write gene names to a text file
with open(output_file_path, 'w') as output_file:
    for gene_name in ref_gene_names:
        output_file.write(f'{gene_name}\n')

output_file_path = 'ref_gene_names.txt'


Then a connection was established to the UniProt REST API in order to retrieve protein data for further analysis.

In [19]:

# UniProt API URL
#url = "https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28brassica+napus%29"
# Send the request to the UniProt API
#response = requests.get(url)
# Check if the request was successful (status code 200)
#if response.status_code == 200:
    # Process the API response (e.g., print the content)
    #print(response.text)
#else:
    # Handle errors
    #print(f"Error: {response.status_code}")



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



The data was imported from the UniProt server with the query as 'organism:Brassica napus'. All the UniProtkb protein entries were written to a TSV file. 

In [27]:
import re
import requests
from requests.adapters import HTTPAdapter, Retry

re_next_link = re.compile(r'<(.+)>; rel="next"')
retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=retries))

def get_next_link(headers):
    if "Link" in headers:
        match = re_next_link.match(headers["Link"])
        if match:
            return match.group(1)

def get_batch(batch_url):
    while batch_url:
        response = session.get(batch_url)
        response.raise_for_status()
        total = response.headers["x-total-results"]
        yield response, total
        batch_url = get_next_link(response.headers)


url = 'https://rest.uniprot.org/uniprotkb/search?fields=accession%2Cprotein_name%2Cgene_names&format=tsv&query=%28brassica+napus%29&size=500'
progress = 0

with open('brassica_napus.tsv', 'w') as f:
    for batch, total in get_batch(url):
        lines = batch.text.splitlines()
        if not progress:
            print(lines[0], file=f)
        for line in lines[1:]:
            print(line, file=f)
        progress += len(lines[1:])
        print(f'{progress} / {total}')

500 / 145683
1000 / 145683
1500 / 145683
2000 / 145683
2500 / 145683
3000 / 145683
3500 / 145683
4000 / 145683
4500 / 145683
5000 / 145683
5500 / 145683
6000 / 145683
6500 / 145683
7000 / 145683
7500 / 145683
8000 / 145683
8500 / 145683
9000 / 145683
9500 / 145683
10000 / 145683
10500 / 145683
11000 / 145683
11500 / 145683
12000 / 145683
12500 / 145683
13000 / 145683
13500 / 145683
14000 / 145683
14500 / 145683
15000 / 145683
15500 / 145683
16000 / 145683
16500 / 145683
17000 / 145683
17500 / 145683
18000 / 145683
18500 / 145683
19000 / 145683
19500 / 145683
20000 / 145683
20500 / 145683
21000 / 145683
21500 / 145683
22000 / 145683
22500 / 145683
23000 / 145683
23500 / 145683
24000 / 145683
24500 / 145683
25000 / 145683
25500 / 145683
26000 / 145683
26500 / 145683
27000 / 145683
27500 / 145683
28000 / 145683
28500 / 145683
29000 / 145683
29500 / 145683
30000 / 145683
30500 / 145683
31000 / 145683
31500 / 145683
32000 / 145683
32500 / 145683
33000 / 145683
33500 / 145683
34000 / 145683


All standard gene identifiers were subsequently mapped to their corresponding UniProt accession identifiers.

In [77]:
def filter_tsv(input_tsv_path, output_tsv_path, keywords):
    # Read the TSV file into a dataframe
    df = pd.read_csv(input_tsv_path, sep='\t')

    # Filter rows containing any of the keywords (case-insensitive)
    filtered_df = df[df['Gene Names'].str.contains('|'.join(keywords), case=False, na=False)]

    # Save the filtered dataframe to a new TSV file
    filtered_df.to_csv(output_tsv_path, sep='\t', index=False)

input_file_path = 'brassica_napus.tsv'
output_file_path = 'matched_output.tsv'
keywords_to_search = ref_gene_names

filter_tsv(input_file_path, output_file_path, keywords_to_search)

For the second half of task 2, our aim was to study enyzmatic activity of Hydrolase proteins on Ester bonds, so the query made to the API used the parameters: 'organism: Brassica napus' and Enzyme Activity(ec:3.1.-.-). The data was similarly written to a TSV file. 

In [90]:
url2 = "https://rest.uniprot.org/uniprotkb/search?fields=accession%2Cprotein_name%2Cgene_names&format=tsv&query=%28%28brassica+napus%29+AND+%28ec%3A3.1.-.-%29%29&size=500"
progress = 0

with open('enzyme_activity.tsv', 'w') as f:
    for batch, total in get_batch(url2):
        lines = batch.text.splitlines()
        if not progress:
            print(lines[0], file=f)
        for line in lines[1:]:
            print(line, file=f)
        progress += len(lines[1:])
        print(f'{progress} / {total}')


500 / 1462
1000 / 1462
1462 / 1462


The genes in specific enzyme activity data were then matched with the genes in aligned sequences from assembly_1 and assembly_2. These proteins will give us an idea of which proteins in assembly_1 are involved in Hydrolase activity on Ester bonds. 

In [154]:
enzyme_file_path = 'enzyme_activity.tsv'
fasta_file_path = 'data/output.fasta'
output_tsv_path = 'required_proteins.tsv'

# Read gene names from enzyme_filtered_file, keeping only genes starting with 'Bna'
enzyme_gene_names = [gene_name for gene_name in pd.read_csv('enzyme_activity.tsv', sep='\t')['Gene Names'].dropna().str.split().explode().tolist() if gene_name.startswith('Bna')]
enzyme_gene_names = set(enzyme_gene_names)

# Read gene names from output.fasta
fasta_gene_names = set(record.description.split()[1] for record in SeqIO.parse(fasta_file_path, "fasta"))

# Find common gene names
common_gene_names = enzyme_gene_names.intersection(fasta_gene_names)

# Read enzyme_file and filter entries based on common gene names
enzyme_df = pd.read_csv(enzyme_file_path, sep='\t')
filtered_entries = enzyme_df[enzyme_df['Gene Names'].str.contains('|'.join(common_gene_names), case=False, na=False)]

# Save the filtered values to a new TSV file containing the required proteins
filtered_df.to_csv(output_tsv_path, sep='\t', index=False)
